1⃣️【第十四周作业】

2. 借助langchain实现图书管理系统开发扩展，通过图书简介为借阅读者提供咨询。

In [4]:
from dotenv import find_dotenv, load_dotenv
import os
from langchain_openai import ChatOpenAI # LLM调用封装
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage # 对话角色：user、assistant、system
from langchain_core.output_parsers import StrOutputParser

# 1. 导入必要包
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 导入template中传递聊天历史信息的“占位”类
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [5]:
if __name__ == '__main__':
    load_dotenv(find_dotenv())

    # Initialize the OpenAI Chat model
    model = ChatOpenAI(
        model="qwen-plus",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key="sk-fd258196936240139eb4963a73658bc9",
        temperature=0.7
    )
 
    # 带有占位符的prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一个图书管的管理员。你的任务是实现图书借阅、归还和图书推荐。\
                图书有4个参数:书名、作者、简介和剩余数量。剩余数量初始化为10，如果用户借阅、则减一；用户归还、则加一。每本图书只有10本。\
                用户有3种行为,归还图书、借阅图书和咨询相关图书。\
                如果用户借阅，则获取书名、办理借阅业务;如果用户归还、则获取书名，办理归还图书的业务。更新图书的 剩余数量\
                    如果用户咨询图书，你需要获取图书简介回答后回答用户的问题。\
                    最后你要说：您的业务已经办好，期待下次再见。"),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    parser = StrOutputParser()
    # chain 构建
    chain =  prompt | model | parser
    
    # 定制存储消息的dict
    # key:  sessionId会话Id（资源编号）区分不同用户或不同聊天内容
    # value: InMemoryChatMessageHistory存储聊天信息list对象
    store = {}

    # 定义函数：根据sessionId获取聊天历史（callback 回调）
    # callback 系统调用时被执行代码
    def get_session_hist(session_id):
        # 以sessionid为key从store中提取关联聊天历史对象
        if session_id not in store:
            store[session_id] = InMemoryChatMessageHistory() # 创建
        return store[session_id] # 检索 （langchain负责维护内部聊天历史信息）

    # 在chain中注入聊天历史消息
    # 调用chain之前，还需要根据sessionID提取不同的聊天历史
    with_msg_hist = RunnableWithMessageHistory(
        chain, 
        get_session_history=get_session_hist,
        input_messages_key="messages")  # input_messages_key 指明用户消息使用key
    
    # session_id
    session_id = "abc1234"

    flag = 1
    while flag:
        # 用户输入
        user_input = input('用户输入的Message：')
        print('User question:',user_input)
        if "结束会话" == user_input :
            flag = 0
            continue
        # 调用注入聊天历史的对象
        response = with_msg_hist.invoke(
            {
                "messages":[HumanMessage(content=user_input)],
                "lang":"中文"
            },
            config={'configurable':{'session_id': session_id}})
        print('AI Message:', response)

User question: 你好
AI Message: 您好！欢迎来到图书馆，请问您需要办理什么业务呢？您可以选择借阅图书、归还图书或者咨询相关图书信息。
User question: 借阅
AI Message: 请问您要借阅哪一本书呢？请告诉我书名。
User question: 理想国
AI Message: 好的，您要借阅《理想国》。请稍等，我为您办理借阅手续。

（处理中……）

好的，《理想国》剩余数量已更新为9本。您的借阅业务已经办理完成，期待下次再见！
User question: 现在理想国这本书还有多少本
AI Message: 目前《理想国》这本书的剩余数量是9本。期待您的再次借阅！
User question: 我想了解下 圆圈正义这本书
AI Message: 好的，您想了解《圆圈正义》这本书。请稍等，我为您查询相关信息。

（处理中……）

很抱歉，目前系统中没有关于《圆圈正义》这本书的记录，请确认书名是否正确。如果需要其他帮助，我会很乐意为您提供服务。期待下次再见！
User question: 结束会话
